<a href="https://colab.research.google.com/github/CnrGames/col_tor2023/blob/main/TD2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Torrent To Google Drive Downloader

**Important Note:** To get more disk space:
> Go to Runtime -> Change Runtime and give GPU as the Hardware Accelerator.  You will get around 384GB to download any torrent you want.

### Install libtorrent and Initialize Session

In [ ]:
!apt install python3-libtorrent

import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

### Mount Google Drive
To stream files we need to mount Google Drive.

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

### Add From Torrent File
You can run this cell to add more files as many times as you want

In [ ]:
from google.colab import files

source = files.upload()
params = {
    "save_path": "/content/drive/My Drive/Torrent",
    "ti": lt.torrent_info(list(source.keys())[0]),
}
downloads.append(ses.add_torrent(params))

### Add From Magnet Link
You can run this cell to add more files as many times as you want

In [1]:
import libtorrent as lt
import time
import os
from google.colab import output

params = {"save_path": "/content/aquiFinalX"}
downloads = []

ses = lt.session()
global count
count = 0

def clear_console():
    # Check if the system is Windows or Unix-based and clear the console accordingly
    if os.name == 'nt':  # for Windows
        os.system('cls')
    else:  # for Unix-based systems (Linux, macOS)
        os.system('clear')

def format_progress(progress, down_speed, up_speed, num_peers):
    progress_str = "{:.2f}%".format(progress * 100)
    down_speed_str = "{:.2f} kB/s".format(down_speed / 1000)
    up_speed_str = "{:.2f} kB/s".format(up_speed / 1000)
    return "Downloading: {} (Down: {} | Up: {} | Peers: {})".format(progress_str, down_speed_str, up_speed_str, num_peers)

def print_progress(handle):
    while not handle.status().is_seeding:
        global count
        clear_console()
        s = handle.status()
        progress_str = format_progress(s.progress, s.download_rate, s.upload_rate, s.num_peers)

        print(progress_str)
        if count == 6:
            output.clear()
            count = 0
        count += 1

        time.sleep(1)
    print("Download complete!")
while True:
    magnet_link = input("Enter Magnet Link /  Exit: ")
    if magnet_link.lower() == "exit":
        break

    handle = lt.add_magnet_uri(ses, magnet_link, params)
    downloads.append(handle)

    # Call the input function only once after adding the torrent


    print("Waiting for download to finish...")
    print_progress(handle)


SyntaxError: ignored

### Start Download
Source: https://stackoverflow.com/a/5494823/7957705 and [#3 issue](https://github.com/FKLC/Torrent-To-Google-Drive-Downloader/issues/3) which refers to this [stackoverflow question](https://stackoverflow.com/a/6053350/7957705)

In [ ]:
import time
from IPython.display import display
import ipywidgets as widgets

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
